<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [123]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [124]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
N_LAG_DAYS = int(secrets.get_secret("N_LAG_DAYS").value)
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

print(location)
print(N_LAG_DAYS)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1162393
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/users/secrets/AQI_API_KEY). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":120002,"usrMsg":"HazelcastInstance[[172.16.4.90]:5900] is not active!","errorMsg":"A runtime error occurred."}', error code: 120002, error msg: A runtime error occurred., user msg: HazelcastInstance[[172.16.4.90]:5900] is not active!

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [114]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [115]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,2.0,finland,Lansi-Turunmaa,Uto,2024-11-19,https://api.waqi.info/feed/@4928


In [116]:
aq_today_df.isna().sum()

pm25       0
country    0
city       0
street     0
date       0
url        0
dtype: int64

In [117]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [126]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.75°N 21.25°E
Elevation 6.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-19,2.80,0.0,7.342588,281.309906,Lansi-Turunmaa
1,2024-11-20,2.55,1.4,71.483337,26.306900,Lansi-Turunmaa
2,2024-11-21,2.55,0.5,40.957813,259.875397,Lansi-Turunmaa
3,2024-11-22,-1.40,0.0,34.576870,255.529663,Lansi-Turunmaa
4,2024-11-23,0.65,0.0,38.936623,287.764954,Lansi-Turunmaa
5,2024-11-24,4.65,0.0,24.868679,202.109497,Lansi-Turunmaa
6,2024-11-25,8.95,0.1,48.159332,211.551331,Lansi-Turunmaa
7,2024-11-26,7.40,0.0,45.912350,241.425491,Lansi-Turunmaa
8,2024-11-27,4.45,0.0,20.240198,321.499329,Lansi-Turunmaa
9,2024-11-28,4.50,0.0,37.342949,250.866257,Lansi-Turunmaa


In [119]:
daily_df.isna().sum()

date                           0
temperature_2m_mean            0
precipitation_sum              0
wind_speed_10m_max             0
wind_direction_10m_dominant    0
city                           0
dtype: int64

In [127]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [122]:
from hsfs.constructor.filter import Filter
from hsfs.feature import Feature

latest_df = air_quality_fg.select_all().filter(Filter(air_quality_fg["date"], Filter.GE, datetime.datetime.now() - datetime.timedelta(days=N_LAG_DAYS))).read()

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1162393/featurestores/query). Server response: 
HTTP code: 500, HTTP reason: Internal Server Error, body: b'{"errorCode":270117,"usrMsg":"Error opening the Hive Metastore client: javax.ejb.EJBTransactionRolledbackException: Exception thrown from bean: javax.ejb.EJBException: com.hazelcast.core.HazelcastInstanceNotActiveException: HazelcastInstance[[172.16.4.90]:5900] is not active!","devMsg":"javax.ejb.EJBTransactionRolledbackException: Exception thrown from bean: javax.ejb.EJBException: com.hazelcast.core.HazelcastInstanceNotActiveException: HazelcastInstance[[172.16.4.90]:5900] is not active!","errorMsg":"Failed to open Hive Metastore client"}', error code: 270117, error msg: Failed to open Hive Metastore client, user msg: Error opening the Hive Metastore client: javax.ejb.EJBTransactionRolledbackException: Exception thrown from bean: javax.ejb.EJBException: com.hazelcast.core.Hazelcast

In [81]:
latest_df

,date,pm25,pm_25_lag_1,pm_25_lag_2,pm_25_lag_3,country,city,street,url
0,2024-11-17 00:00:00+00:00,18.0,15.0,17.0,11.0,finland,Lansi-Turunmaa,Uto,https://api.waqi.info/feed/@4928
1,2024-11-19 00:00:00+00:00,2.0,4.0,18.0,15.0,finland,Lansi-Turunmaa,Uto,https://api.waqi.info/feed/@4928
2,2024-11-18 00:00:00+00:00,4.0,18.0,15.0,17.0,finland,Lansi-Turunmaa,Uto,https://api.waqi.info/feed/@4928


In [84]:
aq_today_df

,pm25,country,city,street,date,url
0,2.0,finland,Lansi-Turunmaa,Uto,2024-11-19,https://api.waqi.info/feed/@4928


In [109]:
latest_df.sort_values(by="date", inplace=True)
for i in range(1, N_LAG_DAYS+1):
	aq_today_df[f"pm_25_lag_{i}"] = latest_df.iloc[-1]["pm25"] if i == 1 else latest_df.iloc[-1][f"pm_25_lag_{i-1}"]

aq_today_df

,pm25,country,city,street,date,url,pm_25_lag_1,pm_25_lag_2,pm_25_lag_3
0,2.0,finland,Lansi-Turunmaa,Uto,2024-11-19,https://api.waqi.info/feed/@4928,2.0,4.0,18.0


In [110]:
air_quality_fg.insert(aq_today_df)

Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1162393/fs/1153096/fg/1357138


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

KafkaException: KafkaError{code=_UNKNOWN_PARTITION,val=-190,str="Failed to get watermark offsets: Local: Unknown partition"}

In [129]:
daily_df

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-19,2.80,0.0,7.342588,281.309906,Lansi-Turunmaa
1,2024-11-20,2.55,1.4,71.483337,26.306900,Lansi-Turunmaa
2,2024-11-21,2.55,0.5,40.957813,259.875397,Lansi-Turunmaa
3,2024-11-22,-1.40,0.0,34.576870,255.529663,Lansi-Turunmaa
4,2024-11-23,0.65,0.0,38.936623,287.764954,Lansi-Turunmaa
5,2024-11-24,4.65,0.0,24.868679,202.109497,Lansi-Turunmaa
6,2024-11-25,8.95,0.1,48.159332,211.551331,Lansi-Turunmaa
7,2024-11-26,7.40,0.0,45.912350,241.425491,Lansi-Turunmaa
8,2024-11-27,4.45,0.0,20.240198,321.499329,Lansi-Turunmaa
9,2024-11-28,4.50,0.0,37.342949,250.866257,Lansi-Turunmaa


In [131]:
# Insert new data
weather_fg.insert(daily_df, write_options={"wait_for_job": True})

Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1162393/fs/1153096/fg/1355110


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1162393/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7a538c3a3920>,
 {
   "results": [
     {
       "expectation_config": {
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "expectation_type": "expect_column_min_to_be_between",
         "meta": {
           "expectationId": 678936
         }
       },
       "result": {
         "observed_value": 7.342587947845459,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T09:05:16.000103Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       },
       "success": true
     },
     {
       "expectation_config": {
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": 

In [132]:
weather_fg.read().sort_values(by="date")

Finished: Reading data from Hopsworks, using ArrowFlight (1.28s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
940,2014-05-05 00:00:00+00:00,4.269333,0.000000,40.743664,338.594116,Lansi-Turunmaa
2407,2014-05-06 00:00:00+00:00,4.302667,0.000000,14.812590,359.689148,Lansi-Turunmaa
244,2014-05-07 00:00:00+00:00,5.158917,0.200000,37.441727,101.778557,Lansi-Turunmaa
993,2014-05-08 00:00:00+00:00,5.404750,4.899999,42.877792,132.347198,Lansi-Turunmaa
3710,2014-05-09 00:00:00+00:00,5.675583,2.600000,28.112288,181.828354,Lansi-Turunmaa
...,...,...,...,...,...,...
3859,2024-11-24 00:00:00+00:00,4.650000,0.000000,24.868679,202.109497,Lansi-Turunmaa
3854,2024-11-25 00:00:00+00:00,8.950000,0.100000,48.159332,211.551331,Lansi-Turunmaa
3853,2024-11-26 00:00:00+00:00,7.400000,0.000000,45.912350,241.425491,Lansi-Turunmaa
3857,2024-11-27 00:00:00+00:00,4.450000,0.000000,20.240198,321.499329,Lansi-Turunmaa


## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
